In [218]:
from sklearn.datasets import make_classification
import pandas as pd 
import numpy as np 

In [258]:
X, y = make_classification(n_samples=400, n_features=4)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4'])
y = pd.Series(y)

Пишем функцию для лучшего разбиения

In [223]:
def entropy(y): 
    y = y.copy()
    p = np.array([x/len(y) for x in np.bincount(y)])  # считаем вероятность вхождения каждого класса в сплит
    p = p[p > 0]                                      # убираем 0 вероятности
    return np.sum(-p*np.log2(p))


In [225]:
def information_gain(y, y_left, y_right): 
    # Размеры выборок
    n, n_left, n_right = len(y), len(y_left), len(y_right)
    
    # Если одна из подвыборок пуста — сплит бесполезен, прирост информации равен 0
    if n_left == 0 or n_right == 0: 
        return 0
    
    # Энтропия исходной выборки (до разделения)
    H_parent = entropy(y)
    
    # Энтропии левой и правой подвыборок (после разделения)
    H_left = entropy(y_left)
    H_right = entropy(y_right)
    
    # Прирост информации = уменьшение энтропии после разбиения
    IG = H_parent - (n_left / n) * H_left - (n_right / n) * H_right
    
    # Возвращаем значение прироста информации
    return IG

    

In [344]:
def get_best_split(X, y):
    # Создаём копии X и y, чтобы не изменять исходные данные
    X, y = X.copy(), y.copy()
   
    # Глобальные переменные для хранения лучшего сплита по всему датасету
    best_IG = -np.inf           # Максимальный прирост информации среди всех колонок
    best_col = None             # Название колонки, по которой достигается лучший сплит
    best_split_value = None     # Значение сплита, которое даёт этот прирост
        
    # Проходим по каждой колонке признаков
    for col in X.columns: 
        # Берём уникальные значения текущей колонки и сортируем их
        unique_values = np.sort(X[col].unique())
        
        # Проходим по всем парам соседних уникальных значений для формирования порогов
        for i in range(len(unique_values)-1): 
            # Порог для сплита — среднее между двумя соседними уникальными значениями
            split = (unique_values[i] + unique_values[i+1]) / 2  
            
            # Создаём булевы маски для левой и правой подвыборки
            left_mask = X[col] <= split
            right_mask = X[col] > split
            
            # Формируем соответствующие подвыборки таргета
            y_left, y_right = y[left_mask], y[right_mask]

            # Вычисляем прирост информации для текущего сплита
            IG = information_gain(y, y_left, y_right)
            
            # Если этот сплит даёт лучший прирост информации среди всех колонок — обновляем глобальные переменные
            if IG > best_IG:
                best_IG = IG
                best_split_value = split
                best_col = col
    
    # Возвращаем кортеж с лучшей колонкой, значением сплита и максимальным приростом информации
    return best_col, best_split_value, best_IG


In [227]:
y = np.array([0,0,1,1])
y_left = np.array([0,0])
y_right = np.array([1,1])

print(information_gain(y, y_left, y_right))  # должно быть ~1.0 (идеальное разделение)


1.0


In [229]:
np.bincount(y,minlength=1)

array([2, 2], dtype=int64)

In [232]:
X.to_numpy().shape

(400, 4)

In [346]:
get_best_split(X, y)

('f4', 0.2736670938170923, 0.564432013341756)

In [347]:
X.shape[1]

4

In [272]:
-np.inf

-inf

True

In [413]:
class MyTreeClf:
    def __init__(self, max_depth=5, min_samples_split=2, max_leafs=20): 
        self.max_depth = max_depth 
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        
    def __repr__(self): 
        return f'MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}'

    def __entropy(self, y): 
        y = y.copy()
        p = np.array([x/len(y) for x in np.bincount(y)])  # считаем вероятность вхождения каждого класса в сплит
        p = p[p > 0]                                      # убираем 0 вероятности
        return np.sum(-p*np.log2(p))

    def __information_gain(self, y, y_left, y_right): 
        # Размеры выборок
        n, n_left, n_right = len(y), len(y_left), len(y_right)
        # Если одна из подвыборок пуста — сплит бесполезен, прирост информации равен 0
        if n_left == 0 or n_right == 0: 
            return 0
        # Энтропия исходной выборки (до разделения)
        H_parent = self.__entropy(y)
    
        # Энтропии левой и правой подвыборок (после разделения)
        H_left = self.__entropy(y_left)
        H_right = self.__entropy(y_right)
    
        # Прирост информации = уменьшение энтропии после разбиения
        IG = H_parent - (n_left / n) * H_left - (n_right / n) * H_right
    
        # Возвращаем значение прироста информации
        return IG    

    def __get_best_split(self, X, y):
        # Создаём копии X и y, чтобы не изменять исходные данные
        X, y = X.copy(), y.copy()
   
        # Глобальные переменные для хранения лучшего сплита по всему датасету
        best_IG = -np.inf           # Максимальный прирост информации среди всех колонок
        best_col = None             # Название колонки, по которой достигается лучший сплит
        best_split_value = None     # Значение сплита, которое даёт этот прирост
        
        # Проходим по каждой колонке признаков
        for col in X.columns: 
            # Берём уникальные значения текущей колонки и сортируем их
            unique_values = np.sort(X[col].unique())
        
            # Проходим по всем парам соседних уникальных значений для формирования порогов
            for i in range(len(unique_values)-1): 
                # Порог для сплита — среднее между двумя соседними уникальными значениями
                split = (unique_values[i] + unique_values[i+1]) / 2  
            
                # Создаём булевы маски для левой и правой подвыборки
                left_mask = X[col] <= split
                right_mask = X[col] > split
            
                # Формируем соответствующие подвыборки таргета
                y_left, y_right = y[left_mask], y[right_mask]

                # Вычисляем прирост информации для текущего сплита
                IG = self.__information_gain(y, y_left, y_right)
            
                # Если этот сплит даёт лучший прирост информации среди всех колонок — обновляем глобальные переменные
                if IG > best_IG:
                    best_IG = IG
                    best_split_value = split
                    best_col = col
    
        # Возвращаем кортеж с лучшей колонкой, значением сплита и максимальным приростом информации
        return best_col, best_split_value, best_IG



In [409]:
a = MyTreeClf()

In [411]:
a.get_split(X, y)

('f4', 0.2736670938170923, 0.564432013341756)